## Variables are not boxes

##### Example 8-1. Variables a and b hold references to the same list, not copies of the list.

In [1]:
a = [1, 2, 3]
b = a
a.append(4)
b

[1, 2, 3, 4]

![variables](variables.JPG)

##### Example 8-2. Variables are assigned to objects only after the objects are created.

In [3]:
class Gizmo:
    def __init__(self):
        print('Gizmo id: %d' % id(self))
x = Gizmo()

Gizmo id: 2161216336448


In [7]:
# y = Gizmo() * 10

#File "<stdin>", line 1, in <module>
#TypeError: unsupported operand type(s) for *: 'Gizmo' and 'int'

Here is proof that a second Gizmo was actually instantiated before the
multiplication was attempted

> To understand an assignment in Python, always read the right-hand
side first: that’s where the object is created or retrieved. After that, the
variable on the left is bound to the object, like a label stuck to it. Just
forget about the boxes.

## Identity, equality and aliases

In [10]:
charles = {'name': 'Charles L. Dodgson', 'born': 1832}

# lewis is an alias for charles.
lewis = charles

# The is operator and the id function confirm it.
lewis is charles

True

In [11]:
id(charles), id(lewis)

(2161220571000, 2161220571000)

In [13]:
# Adding an item to lewis is the same as adding an item to charles.
lewis['balance'] = 950
charles

{'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950}

##### Example 8-4. alex and charles compare equal, but alex is not charles.

In [16]:
# alex refers to an object that is a replica of the object assigned to charles.
alex = {'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950}
# The objects compare equal, because of the __eq__ implementation in the dict class.
alex == charles

True

In [17]:

# But they are distinct objects. This is the Pythonic way of writing the negative identity comparison: a is not b.
alex is charles

False

> lewis and charles are aliases: two
variables bound to the same object. On the other hand, alex is not an alias for
charles: these variables are bound to distinct objects. The objects bound to alex and
charles have the same value — that’s what == compares — but they have different
identities.

## Choosing between == and is

**The `==`operator compares the values of objects (the data they hold), while` is `compares
their identities.**

We often care about values and not identities, so == appears more frequently than is in
Python code.

However, if you are comparing a variable to a singleton, then it makes sense to use is.
By far, the most common case is checking whether a variable is bound to None. This is
the recommended way to do it:
        
- x is None

And the proper way to write its negation is:

- x is not None
        
The is operator is faster than ==, because it cannot be overloaded, so Python does not
have to find and invoke special methods to evaluate it, and computing is as simple as
comparing two integer ids. In contrast, a == b is syntactic sugar for `a.__eq__(b).` The
`__eq__ `method inherited from object compares object ids, so it produces the same
result as is. But most built-in types override __eq__ with more meaningful implementations
that actually take into account the values of the object attributes. Equality may
involve a lot of processing — for example, when comparing large collections or deeply
nested structures.


## Copies are shallow by default

In [18]:
l1 = [3, [55, 44], (7, 8, 9)]

#list(l1) creates a copy of l1
l2 = list(l1)
l2

[3, [55, 44], (7, 8, 9)]

In [21]:
# the copies are equal
l2 == l1

True

In [22]:
# but refer to two different objects.
l2 is l1

False

>**For lists and other mutable sequences, the shortcut l2 = l1[:] also makes a copy**

##### Example 8-8. Bus picks and drops passengers.

In [27]:
class Bus:
    def __init__(self,passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)
            
    def pick(self, name):
        self.passengers.append(name)
    
    def drop(self, name):
        self.passengers.remove(name)

##### Example 8-9. Effects of using copy versus deepcopy

In [35]:
import copy

# Using copy and deepcopy we create three distinct Bus instances.
bus1 = Bus(['Alice', 'Bill', 'Claire', 'David'])
bus2 = copy.copy(bus1)
bus3 = copy.deepcopy(bus1)

id(bus1), id(bus2), id(bus3)

(2161221086288, 2161221086400, 2161221086344)

In [33]:
# After bus1 drops 'Bill', he is also missing from bus2.
bus1.drop('Bill')
bus2.passengers

['Alice', 'Claire', 'David']

In [36]:
id(bus1.passengers), id(bus2.passengers), id(bus3.passengers)

(2161220993928, 2161220993928, 2161216889992)

In [37]:
# bus3 is a deep copy of bus1, so its passengers attribute refers to another list.
bus3.passengers

['Alice', 'Bill', 'Claire', 'David']

## Defensive programming with mutable parameters

##### Example 8-15 is the implementation TwilightBus and an explanation of the problem.

In [38]:
class TwilightBus:
    """A bus model that makes passengers vanish"""
    def __init__(self, passengers=None):
        if passengers is None:              # Here we are careful to create a new empty list when passengers is None.
            self.passengers = []
        else:
            self.passengers = passengers    # However, this assignment makes self.passengers an alias for passengers
            
    def pick(self, name):
        self.passengers.append(name)
        
    def drop(self, name):
        self.passengers.remove(name)       

# When the methods .remove() and .append() are used with self.passengers,we are actually mutating the original list received as argument to the constructor.

```
def __init__(self, passengers=None):
    if passengers is None:
        self.passengers = []
    else:
        self.passengers = list(passengers)
    # Make a copy of the passengers list, or convert it to a list if it’s not one.
```

## del and garbage collection

>The del statement deletes names, not objects. An object may be garbage collected as
result of a del command, but only if the variable deleted holds the last reference to the
object, or if the object becomes unreachable4. Rebinding a variable may also cause the
number of references to an object reach zero, causing its destruction

##### Example 8-16. Watching the end of an object when no more references point to it.

In [44]:
import weakref

s1 = {1, 2, 3}

# s1 and s2 are aliases referring to the same set, {1, 2, 3}.
s2 = s1

"""
This function must not be a bound method the object about to be destroyed or
otherwise hold a reference to it

"""
def bye():
    print('Gone with the wind...')
    
ender = weakref.finalize(s1, bye)

# The .alive attribute is True before the finalize object is called.
ender.alive

Gone with the wind...


True

In [45]:
del s1

#As discussed, del does not delete an object, just a reference to it.
ender.alive

True

In [46]:
s2 = 'spam'

# Rebinding the last reference, s2, makes {1, 2, 3} unreachable. It is destroyed, the bye callback is invoked and ender.alive becomes False.
ender.alive

Gone with the wind...


False

## Weak references

##### Example 8-17. A weak reference is a callable that returns the referenced object or None if the referent is no more.

In [59]:

"""Invoking wref() returns the referenced object, {0, 1}. Because this is a console
session, the result {0, 1} is bound to the _ variable"""
wref()

{0, 1}

In [60]:
"""a_set no longer refers to the {0, 1} set, so its reference count is decreased. But the _ variable still refers to it."""
a_set = {2, 3, 4}
wref()

{0, 1}